In [ ]:
!pip uninstall -y numpy scipy albumentations scikit-image
!pip install numpy==1.26.4 scipy==1.11.4 albumentations==1.3.1 scikit-image==0.21.0


In [1]:
pip install ultralytics --no-deps


Note: you may need to restart the kernel to use updated packages.


In [ ]:
Menu → Restart Session


In [ ]:
!pip install numpy==1.25.2 --upgrade
!pip install scipy==1.11.2 --upgrade


In [ ]:
import albumentations as A
print(A.__version__)


In [ ]:
pip uninstall -y numpy


In [ ]:
pip install numpy==1.26.4


In [ ]:
import cv2
import numpy as np
import albumentations as A

print("✅ OpenCV:", cv2.__version__)
print("✅ NumPy:", np.__version__)
print("✅ Albumentations:", A.__version__)


In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

print("✅ Albumentations", A.__version__)
print("✅ Torch", torch.__version__)

# =========================================================
# ✅ Custom CCTV Augmentation Pipeline (without Mosaic/MixUp)
# =========================================================
train_transform = A.Compose([
    # Geometric transformations
    A.RandomResizedCrop(size=(640, 640), scale=(0.7, 1.0), ratio=(0.75, 1.33), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15,
                       border_mode=cv2.BORDER_CONSTANT, p=0.8),

    # Brightness / contrast / color (CCTV-like lighting)
    A.RandomBrightnessContrast(p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=15, p=0.6),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.4),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),

    # Blur / noise (simulate poor CCTV quality)
    A.MotionBlur(p=0.3),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GaussNoise(var_limit=(5, 25), p=0.3),

    # Dropouts (simulate occlusion or fog)
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.4),

    ToTensorV2()
])

# =========================================================
# ✅ Load Pretrained YOLOv8m Model
# =========================================================
model = YOLO("yolov8m.pt")

# Disable YOLO’s default heavy augmentations but keep light mosaic/mixup
model.overrides['augment'] = True     # Keep augment True so it can still do small mosaic/mixup
model.overrides['rect'] = False
model.overrides['mosaic'] = 0.3       # Small mosaic probability
model.overrides['mixup'] = 0.2        # Small mixup probability
model.overrides['copy_paste'] = 0.0   # Disable copy-paste

# =========================================================
# ✅ Dataset Paths
# =========================================================
data_yaml = "/kaggle/input/violenceweapondetection/data.yaml"  # replace with your dataset yaml
save_dir = "/kaggle/working/yolov8m_cctv_aug_runs"


# =========================================================
# ✅ Hook Albumentations into YOLO dataloader
# =========================================================
def custom_dataloader_hook(dataset):
    """Inject Albumentations transforms into YOLO’s training dataloader."""
    dataset.transforms = train_transform
    return dataset

# Apply Albumentations augmentations AFTER dataloader is ready
model.add_callback("on_fit_epoch_start", lambda trainer: custom_dataloader_hook(trainer.train_loader.dataset))

# =========================================================
# ✅ Train Model
# =========================================================
results = model.train(
    data=data_yaml,
    epochs=150,                # maximum number of epochs
    imgsz=640,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=save_dir,
    name="yolov8m_cctv_aug",
    workers=2,
    lr0=0.001,
    optimizer="SGD",
    pretrained=True,
    verbose=True,
    val=True,
    exist_ok=True,
    patience=15              # stops training if no improvement for 25 epochs
)

# =========================================================
# ✅ Evaluate Metrics
# =========================================================
metrics = model.val()
print("\n📊 Validation Metrics:")
try:
    print(f"Precision: {metrics.box.pr:.4f}")
    print(f"Recall: {metrics.box.re:.4f}")
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP50-95: {metrics.box.map:.4f}")
except Exception as e:
    print("⚠️ Metrics output error:", e)
    print(metrics)

print("\n✅ Fine-tuning completed successfully with CCTV augmentations (small mosaic & mixup)!")


In [ ]:
import shutil

shutil.make_archive(
    "/kaggle/working/yolov8m_final_cctv_aug_runs",   # output zip path (without .zip extension)
    'zip',                                     # format
    "/kaggle/working/yolov8m_cctv_aug_runs"  # folder to zip
)


This For resume the training 


In [ ]:
!pip install ultralytics

In [ ]:
# =========================================================
# ✅ Imports
# =========================================================
import os
import torch
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

print("✅ Albumentations:", A.__version__)
print("✅ Torch:", torch.__version__)

# =========================================================
# ✅ CCTV Albumentations Augmentation Pipeline (Fixed)
# =========================================================
train_transform = A.Compose([
    # Geometry (safe for bounding boxes)
    A.RandomSizedBBoxSafeCrop(height=640, width=640, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.Affine(scale=(0.9, 1.1), translate_percent=(0.05, 0.05), rotate=(-15, 15), p=0.8),

    # Lighting & Color (simulate CCTV variations)
    A.RandomBrightnessContrast(p=0.7),
    A.HueSaturationValue(p=0.6),
    A.CLAHE(p=0.4),
    A.RGBShift(p=0.3),

    # Blur / Noise (simulate low-quality surveillance)
    A.MotionBlur(p=0.3),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GaussNoise(p=0.3),

    # Partial occlusion / smoke / dust effects
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.4),

    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# =========================================================
# ✅ Load YOLO Model from Last Checkpoint (Resume Point)
# =========================================================
last_checkpoint = "/kaggle/input/v9leave/New folder/last (15).pt"
model = YOLO(last_checkpoint)
print(f"✅ Loaded checkpoint: {last_checkpoint}")

# =========================================================
# ✅ Dataset YAML (Same as before)
# =========================================================
data_yaml = "/kaggle/input/violenceweapondetection/data.yaml"

# =========================================================
# ✅ Attach Albumentations to YOLO’s Training Loader
# =========================================================
def apply_albumentations(trainer):
    trainer.train_loader.dataset.transform = train_transform

model.add_callback("on_train_start", apply_albumentations)

# =========================================================
# ✅ Resume Training from Last Epoch
# =========================================================
results = model.train(
    data=data_yaml,
    imgsz=640,
    epochs=150,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu',
    project="/kaggle/working/yolo_cctv_aug_runs",
    name="yolov9m_cctv_aug_resume",
    resume=True,                     # ✅ Continue training from where you stopped
    workers=2,
    val=True,
    exist_ok=True,
    patience=15
)

# =========================================================
# ✅ Validate After Training
# =========================================================
metrics = model.val()

print("\n📊 Final Validation Metrics:")
try:
    print(f"Precision: {metrics.box.pr:.4f}")
    print(f"Recall: {metrics.box.re:.4f}")
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP50-95: {metrics.box.map:.4f}")
except:
    print(metrics)

print("\n✅ Training Resumed Successfully with CCTV Augmentations Active!")
